In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os


data_path = 'data/team_stats_cleaned.csv'

try:
    df = pd.read_csv(data_path)
    print("data read successfully")
    print(df.head())
except FileNotFoundError:
    print("file could not be found")
    df = None

print("\nExcluding data from the 2020 season due to potential outliers.")
df_filtered = df[df['year'] != 2020].copy()

Y = df_filtered['win_percentage'] # our target variable
X = df_filtered.drop(columns=['team','wins', 'year', 'win_percentage'])